In [10]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Lambda, concatenate, Conv2D, MaxPool2D, UpSampling2D, \
    Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers import Adam
import numpy as np
import time

In [2]:
vgg19 = VGG19(input_shape=(224,224,3)) # include_top=True, weights='imagenet'
vgg19.trainable = False
pre224 = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19.layers[2].output)
pre112 = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19.layers[5].output)
pre56  = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19.layers[10].output)
pre28  = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19.layers[15].output)
pre14  = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19.layers[20].output)
concat = concatenate([vgg19.input, pre224, pre112, pre56, pre28, pre14]) # 224,224,1472
x = Conv2D(64, (1,1), padding='same', activation='relu')(concat)
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = MaxPool2D((2,2))(x)
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = UpSampling2D((2,2))(x)
x = Conv2DTranspose(64, (3,3), padding='same', activation='relu')(x)
x = Conv2DTranspose(64, (3,3), padding='same', activation='relu')(x)
x = Conv2D(3, (3,3), padding='same')(x)

modelGen = Model(vgg19.input, x)
modelGen.summary()
modelGen.save("generatorCNN.h5")

###

vgg19HM = VGG19(input_shape=(224,224,3))
vgg19HM.trainable = False
pre224HM = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19HM.layers[2].output)
pre112HM = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19HM.layers[5].output)
pre56HM  = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19HM.layers[10].output)
pre28HM  = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19HM.layers[15].output)
pre14HM  = Lambda(lambda image: tf.image.resize(image, (224,224)))(vgg19HM.layers[20].output)

modelDis = Model(vgg19HM.input, [pre224HM,pre112HM,pre56HM,pre28HM,pre14HM])
modelDis.summary()
modelDis.save("discriminatorCNN.h5")

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

| - | non-eager run | eager run |
| - | - | - |
| declared tensor type | eager | resourceVariable |
| declared tensor shape | cannot be None (batch size of y_true/y_pred)  | can be None (batch size of y_true/y_pred)  |
| compute with non-eager tensor (y_true/y_pred) | no | yes |
| speed | slow | fast |

+ Theory behind Tensorflow - Eager execution vs non-eager excution:
    + reference: https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6#:~:text=Eager%20execution%20is%20slower%20than%20graph%20execution!&text=Since%20eager%20execution%20runs%20all,Graphs%2C%20or%20tf.
    + 28\*28\*3 3-layer CNN inference, batch_size=None, data=100: non-eager 17 sec / eager 27 sec; ~ x1.6
+ Testing by me:
  + 224\*224\*3 15-layer CNN training, batch_size=4, data=8: non-eager 21 sec / eager 35 sec; x1.6
  + training initialize time: non-eager~2, eager<1

In [12]:
# non-eager mode
def myloss(y_true, y_pred, model=modelDis, batch_size=4): # 2 HMs comparison # shape=(batch_size,224,224,3)   
    loss = K.mean( K.square(y_pred-y_true), axis=[1,2,3])
    dis_y_true = modelDis(y_true)        # feature list # shape=(batch_size,n*n*channels)
    dis_y_pred = modelDis(y_pred)
    for i in range( len(dis_y_true) ):   # sum over each discriminator output layer
        loss = loss + K.mean(K.square(dis_y_pred[i]-dis_y_true[i]),axis=[1,2,3]) # += is unavailable
    return loss

modelGen.compile(optimizer="Adam", loss=myloss)
start = time.time()
modelGen.fit(np.random.random((8,224,224,3)), np.random.random((8,224,224,3)), epochs=10, batch_size=4)
print(time.time()-start)

Epoch 1/10
2/2 [==============================] - 1s 444ms/step - loss: 84.3679
Epoch 2/10
2/2 [==============================] - 1s 444ms/step - loss: 102.4954
Epoch 3/10
2/2 [==============================] - 1s 441ms/step - loss: 89.5628
Epoch 4/10
2/2 [==============================] - 1s 448ms/step - loss: 62.2750
Epoch 5/10
2/2 [==============================] - 1s 449ms/step - loss: 56.4600
Epoch 6/10
2/2 [==============================] - 1s 440ms/step - loss: 52.3308
Epoch 7/10
2/2 [==============================] - 1s 441ms/step - loss: 47.9857
Epoch 8/10
2/2 [==============================] - 1s 440ms/step - loss: 46.4781
Epoch 9/10
2/2 [==============================] - 1s 444ms/step - loss: 45.5851
Epoch 10/10
2/2 [==============================] - 1s 441ms/step - loss: 43.9216
21.41653060913086


In [11]:
# eager mode
def myloss(y_true, y_pred, model=modelDis): # 2 HMs comparison # shape=(batch_size,224,224,3)   
    dis_y_true = modelDis(y_true)        # feature list # shape=(batch_size,n*n*channels)
    dis_y_pred = modelDis(y_pred)
    dis_loss = K.zeros(y_pred.shape[0])  # shape=(batch_size,) # run in eager mode # type=resourceVariable
    for i in range( len(dis_y_true) ):   # sum over each discriminator output layer
        dis_loss = dis_loss + K.mean(K.square(dis_y_pred[i]-dis_y_true[i]),axis=[1,2,3]) # += is unavailable
    gen_loss = K.mean( K.square(y_pred-y_true), axis=[1,2,3])  # shape=(batch_size,)
    return dis_loss + gen_loss

modelGen.compile(optimizer="Adam", loss=myloss, run_eagerly=True)
start = time.time()
modelGen.fit(np.random.random((8,224,224,3)), np.random.random((8,224,224,3)), epochs=10, batch_size=4)
print(time.time()-start)

Epoch 1/10
2/2 [==============================] - 2s 772ms/step - loss: 138.3438
Epoch 2/10
2/2 [==============================] - 2s 766ms/step - loss: 82.1366
Epoch 3/10
2/2 [==============================] - 2s 767ms/step - loss: 82.2696
Epoch 4/10
2/2 [==============================] - 2s 771ms/step - loss: 65.8501
Epoch 5/10
2/2 [==============================] - 2s 768ms/step - loss: 58.7501
Epoch 6/10
2/2 [==============================] - 2s 767ms/step - loss: 52.2517
Epoch 7/10
2/2 [==============================] - 2s 768ms/step - loss: 49.5799
Epoch 8/10
2/2 [==============================] - 2s 814ms/step - loss: 47.7226
Epoch 9/10
2/2 [==============================] - 2s 768ms/step - loss: 46.2195
Epoch 10/10
2/2 [==============================] - 2s 769ms/step - loss: 44.6778
31.016108751296997
